In [1]:
pip install transformers

Note: you may need to restart the kernel to use updated packages.


In [2]:
import torch
import transformers
from transformers import BertTokenizer
from IPython.display import clear_output

In [3]:
PRETRAINED_MODEL_NAME = "ckiplab/bert-base-chinese"  # 指定繁簡中文 BERT-BASE 預訓練模型

from transformers import (
   BertTokenizerFast,
   AutoModelForMaskedLM,
   AutoModelForCausalLM,
   AutoModelForTokenClassification,
   AutoModel,
)


tokenizer = BertTokenizerFast.from_pretrained(PRETRAINED_MODEL_NAME)


In [4]:
vocab = tokenizer.vocab
print("字典大小：", len(vocab))

字典大小： 21128


In [5]:
import pandas as pd

In [6]:
#from google.colab import drive
#drive.mount('/content/drive')

In [7]:
df_train=pd.read_csv("novels8_data.csv")
df_val=pd.read_csv("novels8_val.csv")
len_=len(df_train)
novels=["詭秘之主","驚悚樂園","大奉打更人","鬥破蒼穹","仙逆","琥珀之劍","全職高手","修真聊天群"]
print("data len=",len_)
print("val len=",len(df_val))

data len= 1242949
val len= 7844


In [8]:
#-------把太長的去掉------
MAX_LENGTH=80

df_train = df_train[~(df_train.sentence.apply(lambda x : len(x)> MAX_LENGTH))]
df_val = df_val[~(df_val.sentence.apply(lambda x : len(x)> MAX_LENGTH))]
len_=len(df_train)
print("after len=",len_)
print("after len=",len(df_val))

after len= 1170400
after len= 6356


In [9]:
# idempotence, 將處理結果另存成 tsv 供 PyTorch 使用

print("訓練樣本數：", len(df_train))
print(df_train.head())
print(df_val.head())

訓練樣本數： 1170400
                                            sentence label
0                                          作者：愛潛水的烏賊  詭秘之主
1  　　蒸汽與機械的浪潮中，誰能觸及非凡？歷史和黑暗的迷霧裡，又是誰在耳語？我從詭祕中醒來，睜眼...  詭秘之主
2                                 　　槍械，大炮，鉅艦，飛空艇，差分機  詭秘之主
3        魔藥，占卜，詛咒，倒吊人，封印物……光明依舊照耀，神祕從未遠離，這是一段“愚者”的傳說  詭秘之主
4                                             第一部 小丑  詭秘之主
                                         sentence label
0  巴頓的心臟狂跳了起來，努力地想要避開對方，可無論他怎麼掙扎，身體都像凍僵了一樣，完全無法動彈  詭秘之主
1          就在弗納爾的臉部湊到了他的眼睛前方時，巴頓右手忽然一痛，整個人瞬間恢復了清醒  詭秘之主
2                  他眼前的淡白霧氣和怪物般的弗納爾隨之不見，像是從未出現過一樣  詭秘之主
3               巴頓木然低頭，看見右手夾著的那根香菸已燃燒到盡頭，灼傷了自己的手指  詭秘之主
4                            剛才是一場夢？可我感覺，感覺是那樣的真實  詭秘之主


In [10]:
df_train.label.value_counts() / len(df_train)
df_train.to_csv("train.tsv", sep="\t", index=False)
df_train.label.value_counts() / len(df_train)

修真聊天群    0.279804
全職高手     0.138831
大奉打更人    0.121558
驚悚樂園     0.109200
琥珀之劍     0.103901
仙逆       0.098694
詭秘之主     0.092473
鬥破蒼穹     0.055538
Name: label, dtype: float64

In [11]:
df_val.label.value_counts() / len(df_val)
df_val.to_csv("val.tsv", sep="\t", index=False)
df_val.label.value_counts() / len(df_val)

全職高手     0.139396
修真聊天群    0.139396
大奉打更人    0.139081
詭秘之主     0.133732
驚悚樂園     0.121775
鬥破蒼穹     0.114223
琥珀之劍     0.114223
仙逆       0.098175
Name: label, dtype: float64

In [12]:
#if data is unbalance https://stackoverflow.com/questions/57244781/deleting-large-amount-of-data-from-pandas-dataframe
"""
bad_label_index=df_val[df_val["label"]=="血觀音"].sample(60).index
df_val = df_val[~df_val.index.isin(bad_label_index)]
t=df_val.index.isin(bad_label_index)
"""
"""
percentage=df_train.label.value_counts() / len(df_train)
while max(percentage)>0.20:
    for serie in series:
        if percentage[serie]>0.20:
            bad_label_index=df_val[df_val["label"]==serie].sample(len(df_val)//20).index
            df_val = df_val[~df_val.index.isin(bad_label_index)]
    percentage=df_val.label.value_counts() / len(df_val)
print(percentage)
df_val.to_csv("val.tsv", sep="\t", index=False)
"""

'\npercentage=df_train.label.value_counts() / len(df_train)\nwhile max(percentage)>0.20:\n    for serie in series:\n        if percentage[serie]>0.20:\n            bad_label_index=df_val[df_val["label"]==serie].sample(len(df_val)//20).index\n            df_val = df_val[~df_val.index.isin(bad_label_index)]\n    percentage=df_val.label.value_counts() / len(df_val)\nprint(percentage)\ndf_val.to_csv("val.tsv", sep="\t", index=False)\n'

In [13]:
#if data is unbalance https://stackoverflow.com/questions/57244781/deleting-large-amount-of-data-from-pandas-dataframe
"""
bad_label_index=df_val[df_val["label"]=="血觀音"].sample(60).index
df_val = df_val[~df_val.index.isin(bad_label_index)]
t=df_val.index.isin(bad_label_index)
"""
"""
percentage=df_total.label.value_counts() / len(df_total)
while max(percentage)>0.20:
    for serie in series:
        if percentage[serie]>0.20:
            bad_label_index=df_total[df_total["label"]==serie].sample(len(df_total)//20).index
            df_total = df_total[~df_total.index.isin(bad_label_index)]
    percentage=df_total.label.value_counts() / len(df_total)
print(percentage)
df_total.to_csv("total.tsv", sep="\t", index=False)
"""

'\npercentage=df_total.label.value_counts() / len(df_total)\nwhile max(percentage)>0.20:\n    for serie in series:\n        if percentage[serie]>0.20:\n            bad_label_index=df_total[df_total["label"]==serie].sample(len(df_total)//20).index\n            df_total = df_total[~df_total.index.isin(bad_label_index)]\n    percentage=df_total.label.value_counts() / len(df_total)\nprint(percentage)\ndf_total.to_csv("total.tsv", sep="\t", index=False)\n'

tokens_tensor：代表識別每個 token 的索引值，用 tokenizer 轉換即可  

segments_tensor：用來識別句子界限。第一句所有index對應 0，第二句所有index對應 1。另外注意句子間的 [SEP] 為 0  

masks_tensor：用來界定自注意力機制範圍。1 讓 BERT 關注該位置，0 則代表是 padding 不需關注  //因為我們會zero padding讓batch裡大小一致


In [14]:
label_map = {novel:i  for i, novel in enumerate(novels)}
inv_label_map = {v: k for k, v in label_map.items()}
print(inv_label_map)

def indices_to_labels(indices):
    for i in range(len(indices)):
        indices[i] = inv_label_map[indices[i]]
    return indices

print(indices_to_labels([4, 5, 6]))

{0: '詭秘之主', 1: '驚悚樂園', 2: '大奉打更人', 3: '鬥破蒼穹', 4: '仙逆', 5: '琥珀之劍', 6: '全職高手', 7: '修真聊天群'}
['仙逆', '琥珀之劍', '全職高手']


In [15]:
"""
實作一個可以用來讀取訓練 / 測試集的 Dataset，這是你需要徹底了解的部分。
此 Dataset 每次將 tsv 裡的一筆成對句子轉換成 BERT 相容的格式，並回傳 3 個 tensors：
- tokens_tensor：兩個句子合併後的索引序列，包含 [CLS] 與 [SEP]
- segments_tensor：可以用來識別兩個句子界限的 binary tensor
- label_tensor：將分類標籤轉換成類別索引的 tensor, 如果是測試集則回傳 None
"""
from torch.utils.data import Dataset

#https://pytorch.org/tutorials/beginner/data_loading_tutorial.html
class NovelsDataset(Dataset):
    # 讀取前處理後的 tsv 檔並初始化一些參數
    def __init__(self, mode, tokenizer,label_map):
        #assert mode in ["train", "test","val"]  # 確定檔案名稱只接受train或test
        self.mode = mode
        # 大數據你會需要用 iterator=True
        self.df = pd.read_csv(mode + ".tsv", sep="\t").fillna("")   #讀處裡好的tsv避免跟所有檔案的csv搞混
        self.len = len(self.df)
        self.label_map = label_map
        self.tokenizer = tokenizer  # 我們將使用 BERT tokenizer

    # 定義回傳一筆訓練 / 測試數據的函式
    def __getitem__(self, idx):
        """
        if self.mode == "test":
            text, label = self.df.iloc[idx, :].values
            label_tensor = None
        else:
        """
        text,label = self.df.iloc[idx, :].values
        # 將 label 文字也轉換成索引方便轉換成 tensor
        label_id = self.label_map[label]
        label_tensor = torch.tensor(label_id)

        # 建立第一個句子的 BERT tokens 並加入分隔符號 [SEP]
        word_pieces = ["[CLS]"]
        tokens = self.tokenizer.tokenize(text)
        word_pieces += tokens + ["[SEP]"]
        
        # 將整個 token 序列轉換成索引序列
        ids = self.tokenizer.convert_tokens_to_ids(word_pieces)
        tokens_tensor = torch.tensor(ids)

        return (tokens_tensor, label_tensor)

    def __len__(self):
        return self.len

In [16]:
# 初始化一個專門讀取訓練樣本的 Dataset，使用中文 BERT 斷詞
trainset = NovelsDataset("train", tokenizer=tokenizer,label_map=label_map)     #tokenizer 在上面注意力圖 上有宣告
valset=NovelsDataset("val", tokenizer=tokenizer,label_map=label_map)
#totalset=NovelsDataset("total", tokenizer=tokenizer,label_map=label_map)
#valset_replace = SeriesDataset("val_replace", tokenizer=tokenizer,label_map=label_map)     #tokenizer 在上面注意力圖 上有宣告
print(trainset.df)
print(valset.df)
#print(valset_replace.df)

                                                  sentence  label
0                                                作者：愛潛水的烏賊   詭秘之主
1        　　蒸汽與機械的浪潮中，誰能觸及非凡？歷史和黑暗的迷霧裡，又是誰在耳語？我從詭祕中醒來，睜眼...   詭秘之主
2                                       　　槍械，大炮，鉅艦，飛空艇，差分機   詭秘之主
3              魔藥，占卜，詛咒，倒吊人，封印物……光明依舊照耀，神祕從未遠離，這是一段“愚者”的傳說   詭秘之主
4                                                   第一部 小丑   詭秘之主
...                                                    ...    ...
1170395                                        　　恆火尊者：“……”  修真聊天群
1170396                             　　“而且，三浪前輩只是做回了真正的自己而已  修真聊天群
1170397                    ”宋書航又道：“這一次，三浪前輩他只為自己而活，不為其他任何人  修真聊天群
1170398         　　而恰好的，吸引雲雀子仙子的，是如今這個單純的‘狂刀三浪’個體，不包含任何其他因素  修真聊天群
1170399                    　　單純只是因為他是三浪，所以才顯得有魅力，吸引著所有人的注意  修真聊天群

[1170400 rows x 2 columns]
                                               sentence  label
0        巴頓的心臟狂跳了起來，努力地想要避開對方，可無論他怎麼掙扎，身體都像凍僵了一樣，完全無法動彈   詭秘之主
1                就在弗納爾的臉部湊到了他的眼睛前方時，巴頓右手忽然一痛，整個人瞬間恢復了清

In [17]:
import random
# 選擇一個樣本
sample_idx = int(random.random()*len(trainset))
print(sample_idx)
# 將原始文本拿出做比較
text,label = trainset.df.iloc[sample_idx]

# 利用剛剛建立的 Dataset 取出轉換後的 id tensors
tokens_tensor,label_tensor = trainset[sample_idx]

# 將 tokens_tensor 還原成文本
tokens = tokenizer.convert_ids_to_tokens(tokens_tensor.tolist())
combined_text = "".join(tokens)     #tokens 是一個很多字元的list用.join把他轉成string

# 渲染前後差異，毫無反應就是個 print。可以直接看輸出結果
print(f"""[原始文本]
句子 1：{text}
分類  ：{label}

--------------------

[Dataset 回傳的 tensors]
tokens_tensor  ：{tokens_tensor}

label_tensor   ：{label_tensor}
--------------------

[還原 tokens_tensors]
{combined_text}
""")

219945
[原始文本]
句子 1：　　數分鐘後，蒜頭哥他們的小船回來了，而封不覺則已經用月步去沙灘那兒走了個來回
分類  ：驚悚樂園

--------------------

[Dataset 回傳的 tensors]
tokens_tensor  ：tensor([ 101, 3149, 1146, 7132, 2527, 8024, 5886, 7531, 1520,  800,  947, 4638,
        2207, 5670, 1726,  889,  749, 8024, 5445, 2196,  679, 6221, 1179, 2347,
        5195, 4500, 3299, 3635, 1343, 3763, 4121, 6929, 1051, 6624,  749,  943,
         889, 1726,  102])

label_tensor   ：1
--------------------

[還原 tokens_tensors]
[CLS]數分鐘後，蒜頭哥他們的小船回來了，而封不覺則已經用月步去沙灘那兒走了個來回[SEP]



In [18]:
print(type(trainset[0]))
print(len(trainset))
print(type(trainset))

#trainset 直接用[]來看是一個(2657,)個tuple of three tensor

<class 'tuple'>
1170400
<class '__main__.NovelsDataset'>


In [19]:
"""
實作可以一次回傳一個 mini-batch 的 DataLoader
這個 DataLoader 吃我們上面定義的 `MovieDataset`，
回傳訓練 BERT 時會需要的 4 個 tensors：
- tokens_tensors  : (batch_size, max_seq_len_in_batch)
- segments_tensors: (batch_size, max_seq_len_in_batch)
- masks_tensors   : (batch_size, max_seq_len_in_batch)
- label_ids       : (batch_size)
"""

from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence

# 這個函式的輸入 `samples` 是一個 list由前面的3個tenors組成，裡頭的每個 element 都是
# 剛剛定義的 `MovieDataset` 回傳的一個樣本，每個樣本都包含 3 tensors：
# - tokens_tensor
# - segments_tensor
# - label_tensor
# 它會對前兩個 tensors 作 zero padding，並產生前面說明過的 masks_tensors
def create_mini_batch(samples):
    tokens_tensors = [s[0] for s in samples]       #把每個sample的tensor取出來放進list 產生一個 list of tensor (batch_size,)
    
    # 測試集有 labels
    if samples[0][1] is not None:
        label_ids = torch.stack([s[1] for s in samples])
    else:
        label_ids = None
    
    # zero pad 到同一序列長度(要同時做好幾個RNN就要batch中每個元素一樣長)
    #https://blog.csdn.net/qq_43391414/article/details/123289492
    
    tokens_tensors = pad_sequence(tokens_tensors, 
                                  batch_first=True)
    
    return tokens_tensors, label_ids


In [20]:
#把data跟寫好的create_mini_batch放進trainloader
BATCH_SIZE = 32
trainloader = DataLoader(trainset, batch_size=BATCH_SIZE,
                         shuffle=True,
                         collate_fn=create_mini_batch
                         )

valloader = DataLoader(valset, batch_size=8,
                         shuffle=True,
                         collate_fn=create_mini_batch
                         )
"""
totalloader = DataLoader(valset, batch_size=8,
                         shuffle=True,
                         collate_fn=create_mini_batch
                         )

valloader_replace = DataLoader(valset_replace, batch_size=BATCH_SIZE,
                         shuffle=True,
                         collate_fn=create_mini_batch
                         )
"""

'\ntotalloader = DataLoader(valset, batch_size=8,\n                         shuffle=True,\n                         collate_fn=create_mini_batch\n                         )\n\nvalloader_replace = DataLoader(valset_replace, batch_size=BATCH_SIZE,\n                         shuffle=True,\n                         collate_fn=create_mini_batch\n                         )\n'

In [21]:
data = next(iter(trainloader))  # 取出traing loader 裡的一個mini_batch

tokens_tensors, label_ids = data

print(f"""
tokens_tensors.shape   = {tokens_tensors.shape} 
{tokens_tensors}
------------------------
label_ids.shape        = {label_ids.shape}
{label_ids}
""")


tokens_tensors.shape   = torch.Size([32, 66]) 
tensor([[ 101, 2769, 2682,  ...,    0,    0,    0],
        [ 101,  100, 4692,  ...,    0,    0,    0],
        [ 101, 2129, 2406,  ...,    0,    0,    0],
        ...,
        [ 101,  100, 6929,  ...,    0,    0,    0],
        [ 101, 7350, 4273,  ...,    0,    0,    0],
        [ 101, 1086, 2682,  ...,    0,    0,    0]])
------------------------
label_ids.shape        = torch.Size([32])
tensor([5, 3, 7, 7, 6, 3, 5, 7, 3, 4, 6, 1, 1, 4, 1, 4, 5, 4, 6, 6, 7, 5, 7, 2,
        5, 3, 7, 0, 7, 7, 0, 7])



建立 BERT 用的 mini-batch 時最需要注意的就是 zero padding 的存在了。  
你可以發現除了 lable_ids 以外，其他 3 個 tensors 的每個樣本的最後大都為 0，  
這是因為每個樣本的 tokens 序列基本上長度都會不同，需要補 padding。

In [22]:
from transformers import BertForSequenceClassification

In [23]:
import os
PRETRAINED_MODEL_NAME = "ckiplab/bert-base-chinese"
NUM_LABELS = len(label_map)
PATH="model_novels.pt"

model = BertForSequenceClassification.from_pretrained(
        PRETRAINED_MODEL_NAME, num_labels=NUM_LABELS,attention_probs_dropout_prob=0.10,
        hidden_dropout_prob=0.10)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
#uncommend to load model

if os.path.isfile(PATH):
    print("=====>LOAD checkpoint")
    checkpoint = torch.load(PATH)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
else:
    print("======>LOAD new model")


Some weights of the model checkpoint at ckiplab/bert-base-chinese were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ckiplab/bert-base-c

======>LOAD new model


In [24]:
print("""
name            module
----------------------""")
for name, module in model.named_children():
    if name == "bert":
        for n, _ in module.named_children():
            print(f"{name}:{n}")
    else:
        print("{:15} {}".format(name, module))


name            module
----------------------
bert:embeddings
bert:encoder
bert:pooler
dropout         Dropout(p=0.1, inplace=False)
classifier      Linear(in_features=768, out_features=8, bias=True)


In [25]:
""""
定義一個可以針對特定 DataLoader 取得模型預測結果以及分類準確度的函式
之後也可以用來生成上傳到 Kaggle 競賽的預測結果

2019/11/22 更新：在將 `tokens`、`segments_tensors` 等 tensors
丟入模型時，強力建議指定每個 tensor 對應的參數名稱，以避免 HuggingFace
更新 repo 程式碼並改變參數順序時影響到我們的結果。
"""

def get_predictions(model, dataloader, compute_acc=False):
    predictions = None
    correct = 0
    total = 0
      
    with torch.no_grad():
        # 遍巡整個資料集
        wrong_list=[]
        for data in dataloader:
            # 將所有 tensors 移到 GPU 上
            if next(model.parameters()).is_cuda:     #檢查model是否在GPU
                data = [t.to("cuda:0") for t in data if t is not None]    #sentence 跟label tensor從tuple改放到cuda list
            tokens_tensors= data[0]   #data 已經放在cuda了 如果沒有的話 應該會報錯
            outputs = model(input_ids=tokens_tensors)
            #ouputs 原本是 SequenceClassifierOutput class
            #用outputs[0]可以取出他的tensor部分 shape=(64,3)
            logits = outputs[0]
            _, pred = torch.max(logits.data, 1)   #後面的如果是0回傳整column中最大的,1的話回傳整row最大的
 
            # 用來計算訓練集的分類準確率
            if compute_acc:
                labels = data[1]
                total += labels.size(0)
                correct += (pred == labels).sum().item()   #.item把tensor 轉為純量
                
                cur=0
                for i in (pred!=labels):
                    if i==True:
                        wrong_list.append((tokens_tensors[cur],pred[cur].item(),labels[cur].item()))
                    cur+=1
                
            # 將當前 batch 記錄下來
            if predictions is None:
                predictions = pred
            else:
                predictions = torch.cat((predictions, pred))  #最後會形成一長串所有的prediction
    
    if compute_acc:
        acc = correct / total
        return wrong_list, acc
    return predictions
    

In [26]:

# 讓模型跑在 GPU 上並取得訓練集的分類準確率
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("device:", device)
model = model.to(device)     #把model移到devive 應該會是cuda:0
acc_record =0.5 #set 0 to start saving model
train_acc_list=[]
val_acc_list=[]

device: cuda:0


In [27]:
"""
_, acc = get_predictions(model, trainloader, compute_acc=True)
print("classification acc:", acc)
#print("total prediction:",_.shape)
"""

'\n_, acc = get_predictions(model, trainloader, compute_acc=True)\nprint("classification acc:", acc)\n#print("total prediction:",_.shape)\n'

In [28]:
#for data in trainloader:
    #print(data[0].shape)
print(next(model.parameters()).shape)

torch.Size([21128, 768])


In [29]:
data = next(iter(trainloader))    #並沒有紀錄iter 所以不會一直next下去 只會取trainloader的開頭 並取得next
for t in data:
    print(t.shape)
tmp=[t.to("cuda:0") for t in data if t is not None]
print(type(tmp))
print(type(data))

torch.Size([32, 78])
torch.Size([32])
<class 'list'>
<class 'tuple'>


输入：
    input_ids：训练集，torch.LongTensor类型，shape是[batch_size, sequence_length]
    token_type_ids：可选项，当训练集是两句话时才有的。
    attention_mask：可选项，当使用mask才有，可参考原论文。
    labels：数据标签，torch.LongTensor类型，shape是[batch_size]

In [30]:
%%time

# 訓練模式
model.train()

# 使用 Adam Optim 更新整個分類模型的參數
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5) #1e-4 train不起來


EPOCHS = 1
FREQ=10
for epoch in range(EPOCHS):

    running_loss = 0.0
    running_correct=0
    running_count=0
    model.train()
    
    for i,data in enumerate(trainloader):

        tokens_tensors, labels = [t.to(device) for t in data]

        # 將參數梯度歸零
        optimizer.zero_grad()

        # forward pass
        outputs = model(input_ids=tokens_tensors,
                        labels=labels)

        loss = outputs[0]
        logits=outputs[1]
        
        _, pred = torch.max(logits.data, 1)   #後面的如果是0回傳整column中最大的,1的話回傳整row最大的
        correct = (pred == labels).sum().item()   #.item把tensor 轉為純量
        if i%FREQ==0:            
            print("epoch={} iteration={} acc={}".format(epoch,i,correct/labels.size(0)))
        # backward
        loss.backward()
        optimizer.step()

        # 紀錄當前 batch loss
        running_loss += loss.item()
        running_count+= labels.size(0)
        running_correct+=correct
    # 計算分類準確率
    model.eval()
    acc=running_correct/running_count
    with torch.no_grad():
        _, val_acc = get_predictions(model, valloader, compute_acc=True)
    train_acc_list.append(acc)
    val_acc_list.append(val_acc)
    print('[epoch %d] loss: %.3f, acc: %.3f,val acc: %.3f'%
          (epoch + 1, running_loss, acc, val_acc))


torch.save({
'model_state_dict': model.state_dict(),
'optimizer_state_dict': optimizer.state_dict(),
}, PATH)


epoch=0 iteration=0 acc=0.34375
epoch=0 iteration=10 acc=0.15625
epoch=0 iteration=20 acc=0.34375
epoch=0 iteration=30 acc=0.3125
epoch=0 iteration=40 acc=0.21875
epoch=0 iteration=50 acc=0.25
epoch=0 iteration=60 acc=0.34375
epoch=0 iteration=70 acc=0.25
epoch=0 iteration=80 acc=0.46875
epoch=0 iteration=90 acc=0.5625
epoch=0 iteration=100 acc=0.59375
epoch=0 iteration=110 acc=0.59375
epoch=0 iteration=120 acc=0.625
epoch=0 iteration=130 acc=0.59375
epoch=0 iteration=140 acc=0.5
epoch=0 iteration=150 acc=0.75
epoch=0 iteration=160 acc=0.625
epoch=0 iteration=170 acc=0.6875
epoch=0 iteration=180 acc=0.59375
epoch=0 iteration=190 acc=0.53125
epoch=0 iteration=200 acc=0.6875
epoch=0 iteration=210 acc=0.65625
epoch=0 iteration=220 acc=0.78125
epoch=0 iteration=230 acc=0.625
epoch=0 iteration=240 acc=0.71875
epoch=0 iteration=250 acc=0.6875
epoch=0 iteration=260 acc=0.59375
epoch=0 iteration=270 acc=0.78125
epoch=0 iteration=280 acc=0.71875
epoch=0 iteration=290 acc=0.71875
epoch=0 iterati

epoch=0 iteration=2430 acc=0.90625
epoch=0 iteration=2440 acc=0.875
epoch=0 iteration=2450 acc=0.9375
epoch=0 iteration=2460 acc=0.84375
epoch=0 iteration=2470 acc=0.78125
epoch=0 iteration=2480 acc=0.8125
epoch=0 iteration=2490 acc=0.8125
epoch=0 iteration=2500 acc=0.84375
epoch=0 iteration=2510 acc=0.71875
epoch=0 iteration=2520 acc=0.90625
epoch=0 iteration=2530 acc=0.65625
epoch=0 iteration=2540 acc=0.875
epoch=0 iteration=2550 acc=0.8125
epoch=0 iteration=2560 acc=0.84375
epoch=0 iteration=2570 acc=0.90625
epoch=0 iteration=2580 acc=1.0
epoch=0 iteration=2590 acc=0.8125
epoch=0 iteration=2600 acc=0.9375
epoch=0 iteration=2610 acc=0.84375
epoch=0 iteration=2620 acc=0.78125
epoch=0 iteration=2630 acc=0.78125
epoch=0 iteration=2640 acc=0.875
epoch=0 iteration=2650 acc=0.90625
epoch=0 iteration=2660 acc=0.875
epoch=0 iteration=2670 acc=0.9375
epoch=0 iteration=2680 acc=0.875
epoch=0 iteration=2690 acc=0.90625
epoch=0 iteration=2700 acc=0.90625
epoch=0 iteration=2710 acc=0.84375
epoch=

epoch=0 iteration=4830 acc=0.9375
epoch=0 iteration=4840 acc=0.9375
epoch=0 iteration=4850 acc=0.875
epoch=0 iteration=4860 acc=0.9375
epoch=0 iteration=4870 acc=0.875
epoch=0 iteration=4880 acc=0.875
epoch=0 iteration=4890 acc=0.75
epoch=0 iteration=4900 acc=0.875
epoch=0 iteration=4910 acc=0.6875
epoch=0 iteration=4920 acc=0.9375
epoch=0 iteration=4930 acc=0.84375
epoch=0 iteration=4940 acc=0.875
epoch=0 iteration=4950 acc=0.9375
epoch=0 iteration=4960 acc=0.8125
epoch=0 iteration=4970 acc=0.875
epoch=0 iteration=4980 acc=0.84375
epoch=0 iteration=4990 acc=0.875
epoch=0 iteration=5000 acc=0.875
epoch=0 iteration=5010 acc=0.84375
epoch=0 iteration=5020 acc=0.90625
epoch=0 iteration=5030 acc=0.8125
epoch=0 iteration=5040 acc=0.9375
epoch=0 iteration=5050 acc=0.84375
epoch=0 iteration=5060 acc=0.75
epoch=0 iteration=5070 acc=0.90625
epoch=0 iteration=5080 acc=0.8125
epoch=0 iteration=5090 acc=0.875
epoch=0 iteration=5100 acc=0.96875
epoch=0 iteration=5110 acc=0.90625
epoch=0 iteration=5

epoch=0 iteration=7230 acc=0.90625
epoch=0 iteration=7240 acc=0.875
epoch=0 iteration=7250 acc=0.78125
epoch=0 iteration=7260 acc=0.875
epoch=0 iteration=7270 acc=0.875
epoch=0 iteration=7280 acc=0.90625
epoch=0 iteration=7290 acc=0.875
epoch=0 iteration=7300 acc=0.9375
epoch=0 iteration=7310 acc=0.875
epoch=0 iteration=7320 acc=0.96875
epoch=0 iteration=7330 acc=0.6875
epoch=0 iteration=7340 acc=0.84375
epoch=0 iteration=7350 acc=0.875
epoch=0 iteration=7360 acc=0.78125
epoch=0 iteration=7370 acc=0.875
epoch=0 iteration=7380 acc=0.78125
epoch=0 iteration=7390 acc=0.84375
epoch=0 iteration=7400 acc=0.9375
epoch=0 iteration=7410 acc=0.90625
epoch=0 iteration=7420 acc=0.90625
epoch=0 iteration=7430 acc=0.90625
epoch=0 iteration=7440 acc=0.875
epoch=0 iteration=7450 acc=0.96875
epoch=0 iteration=7460 acc=0.875
epoch=0 iteration=7470 acc=0.90625
epoch=0 iteration=7480 acc=0.90625
epoch=0 iteration=7490 acc=0.875
epoch=0 iteration=7500 acc=0.84375
epoch=0 iteration=7510 acc=0.90625
epoch=0 

epoch=0 iteration=9630 acc=0.90625
epoch=0 iteration=9640 acc=0.875
epoch=0 iteration=9650 acc=0.96875
epoch=0 iteration=9660 acc=0.9375
epoch=0 iteration=9670 acc=0.875
epoch=0 iteration=9680 acc=0.9375
epoch=0 iteration=9690 acc=0.875
epoch=0 iteration=9700 acc=0.90625
epoch=0 iteration=9710 acc=0.90625
epoch=0 iteration=9720 acc=0.8125
epoch=0 iteration=9730 acc=0.96875
epoch=0 iteration=9740 acc=0.84375
epoch=0 iteration=9750 acc=0.84375
epoch=0 iteration=9760 acc=0.9375
epoch=0 iteration=9770 acc=0.875
epoch=0 iteration=9780 acc=0.75
epoch=0 iteration=9790 acc=0.96875
epoch=0 iteration=9800 acc=0.90625
epoch=0 iteration=9810 acc=0.90625
epoch=0 iteration=9820 acc=0.8125
epoch=0 iteration=9830 acc=0.875
epoch=0 iteration=9840 acc=0.875
epoch=0 iteration=9850 acc=0.9375
epoch=0 iteration=9860 acc=1.0
epoch=0 iteration=9870 acc=0.875
epoch=0 iteration=9880 acc=0.9375
epoch=0 iteration=9890 acc=0.90625
epoch=0 iteration=9900 acc=0.9375
epoch=0 iteration=9910 acc=0.90625
epoch=0 iterat

...

epoch=0 iteration=25970 acc=0.9375
epoch=0 iteration=25980 acc=0.78125
epoch=0 iteration=25990 acc=0.875
epoch=0 iteration=26000 acc=0.9375
epoch=0 iteration=26010 acc=0.84375
epoch=0 iteration=26020 acc=0.90625
epoch=0 iteration=26030 acc=0.9375
epoch=0 iteration=26040 acc=0.9375
epoch=0 iteration=26050 acc=0.90625
epoch=0 iteration=26060 acc=0.875
epoch=0 iteration=26070 acc=0.90625
epoch=0 iteration=26080 acc=0.90625
epoch=0 iteration=26090 acc=0.9375
epoch=0 iteration=26100 acc=0.96875
epoch=0 iteration=26110 acc=0.84375
epoch=0 iteration=26120 acc=1.0
epoch=0 iteration=26130 acc=0.96875
epoch=0 iteration=26140 acc=0.9375
epoch=0 iteration=26150 acc=0.84375
epoch=0 iteration=26160 acc=0.78125
epoch=0 iteration=26170 acc=0.96875
epoch=0 iteration=26180 acc=0.96875
epoch=0 iteration=26190 acc=0.9375
epoch=0 iteration=26200 acc=0.96875
epoch=0 iteration=26210 acc=1.0
epoch=0 iteration=26220 acc=1.0
epoch=0 iteration=26230 acc=0.90625
epoch=0 iteration=26240 acc=0.90625
epoch=0 iterati

epoch=0 iteration=28300 acc=0.90625
epoch=0 iteration=28310 acc=0.9375
epoch=0 iteration=28320 acc=0.9375
epoch=0 iteration=28330 acc=0.9375
epoch=0 iteration=28340 acc=0.78125
epoch=0 iteration=28350 acc=0.9375
epoch=0 iteration=28360 acc=1.0
epoch=0 iteration=28370 acc=0.875
epoch=0 iteration=28380 acc=0.96875
epoch=0 iteration=28390 acc=0.875
epoch=0 iteration=28400 acc=0.8125
epoch=0 iteration=28410 acc=0.96875
epoch=0 iteration=28420 acc=0.96875
epoch=0 iteration=28430 acc=1.0
epoch=0 iteration=28440 acc=0.9375
epoch=0 iteration=28450 acc=0.96875
epoch=0 iteration=28460 acc=0.96875
epoch=0 iteration=28470 acc=0.96875
epoch=0 iteration=28480 acc=0.875
epoch=0 iteration=28490 acc=0.96875
epoch=0 iteration=28500 acc=0.96875
epoch=0 iteration=28510 acc=0.96875
epoch=0 iteration=28520 acc=0.9375
epoch=0 iteration=28530 acc=1.0
epoch=0 iteration=28540 acc=0.90625
epoch=0 iteration=28550 acc=0.9375
epoch=0 iteration=28560 acc=0.8125
epoch=0 iteration=28570 acc=0.90625
epoch=0 iteration=2

epoch=0 iteration=30640 acc=0.8125
epoch=0 iteration=30650 acc=0.9375
epoch=0 iteration=30660 acc=0.90625
epoch=0 iteration=30670 acc=0.84375
epoch=0 iteration=30680 acc=0.9375
epoch=0 iteration=30690 acc=0.9375
epoch=0 iteration=30700 acc=1.0
epoch=0 iteration=30710 acc=0.90625
epoch=0 iteration=30720 acc=0.90625
epoch=0 iteration=30730 acc=0.875
epoch=0 iteration=30740 acc=0.9375
epoch=0 iteration=30750 acc=0.9375
epoch=0 iteration=30760 acc=0.96875
epoch=0 iteration=30770 acc=0.78125
epoch=0 iteration=30780 acc=0.96875
epoch=0 iteration=30790 acc=0.96875
epoch=0 iteration=30800 acc=0.9375
epoch=0 iteration=30810 acc=0.8125
epoch=0 iteration=30820 acc=0.90625
epoch=0 iteration=30830 acc=0.84375
epoch=0 iteration=30840 acc=0.875
epoch=0 iteration=30850 acc=0.90625
epoch=0 iteration=30860 acc=0.9375
epoch=0 iteration=30870 acc=0.9375
epoch=0 iteration=30880 acc=0.875
epoch=0 iteration=30890 acc=0.9375
epoch=0 iteration=30900 acc=0.96875
epoch=0 iteration=30910 acc=0.9375
epoch=0 iterat

epoch=0 iteration=32970 acc=0.875
epoch=0 iteration=32980 acc=0.9375
epoch=0 iteration=32990 acc=0.96875
epoch=0 iteration=33000 acc=0.96875
epoch=0 iteration=33010 acc=0.9375
epoch=0 iteration=33020 acc=0.90625
epoch=0 iteration=33030 acc=1.0
epoch=0 iteration=33040 acc=0.96875
epoch=0 iteration=33050 acc=0.90625
epoch=0 iteration=33060 acc=0.875
epoch=0 iteration=33070 acc=0.96875
epoch=0 iteration=33080 acc=0.90625
epoch=0 iteration=33090 acc=0.90625
epoch=0 iteration=33100 acc=0.9375
epoch=0 iteration=33110 acc=0.84375
epoch=0 iteration=33120 acc=0.90625
epoch=0 iteration=33130 acc=0.78125
epoch=0 iteration=33140 acc=0.96875
epoch=0 iteration=33150 acc=0.875
epoch=0 iteration=33160 acc=0.9375
epoch=0 iteration=33170 acc=0.875
epoch=0 iteration=33180 acc=0.96875
epoch=0 iteration=33190 acc=0.9375
epoch=0 iteration=33200 acc=0.96875
epoch=0 iteration=33210 acc=0.9375
epoch=0 iteration=33220 acc=0.96875
epoch=0 iteration=33230 acc=0.875
epoch=0 iteration=33240 acc=0.9375
epoch=0 itera

epoch=0 iteration=35310 acc=0.96875
epoch=0 iteration=35320 acc=1.0
epoch=0 iteration=35330 acc=0.96875
epoch=0 iteration=35340 acc=0.90625
epoch=0 iteration=35350 acc=0.90625
epoch=0 iteration=35360 acc=0.9375
epoch=0 iteration=35370 acc=1.0
epoch=0 iteration=35380 acc=0.8125
epoch=0 iteration=35390 acc=0.96875
epoch=0 iteration=35400 acc=0.9375
epoch=0 iteration=35410 acc=0.84375
epoch=0 iteration=35420 acc=0.90625
epoch=0 iteration=35430 acc=0.875
epoch=0 iteration=35440 acc=0.9375
epoch=0 iteration=35450 acc=0.875
epoch=0 iteration=35460 acc=0.9375
epoch=0 iteration=35470 acc=0.84375
epoch=0 iteration=35480 acc=0.9375
epoch=0 iteration=35490 acc=0.90625
epoch=0 iteration=35500 acc=0.8125
epoch=0 iteration=35510 acc=0.9375
epoch=0 iteration=35520 acc=0.9375
epoch=0 iteration=35530 acc=0.875
epoch=0 iteration=35540 acc=0.96875
epoch=0 iteration=35550 acc=0.96875
epoch=0 iteration=35560 acc=0.90625
epoch=0 iteration=35570 acc=0.9375
epoch=0 iteration=35580 acc=0.9375
epoch=0 iteration

In [31]:
#print wrong predict of val
model.eval()
wrong_list, acc = get_predictions(model, valloader, compute_acc=True)
print(acc)
print(wrong_list[0])
for wrong in wrong_list:
    tokens = tokenizer.convert_ids_to_tokens(wrong[0])
    combined_text = "".join(tokens)     #tokens 是一個很多字元的list用.join把他轉成string
    combined_text=combined_text.replace("[PAD]","")
    print(combined_text)
    print("predict:{} true:{}".format(inv_label_map[wrong[1]],inv_label_map[wrong[2]]))

0.7528319697923223
(tensor([ 101, 3297, 2527, 3221, 6283, 6341,  671, 6341, 6857, 3315, 3292, 6279,
        4530, 1168, 3513, 7953, 4638, 5023, 5159, 6257, 2137, 8024, 2682, 2769,
        4534, 1159, 6917, 6291, 4696, 4777, 4955,  671,  678,  800, 4638, 6211,
        1179, 8024, 5439, 2179, 6303, 8024, 1762, 6486, 2134,  722, 1184, 8024,
        6486, 4788, 4638, 5023, 5159, 6917, 3221, 2523, 3300, 6211, 1179, 4638,
         102], device='cuda:0'), 1, 3)
[CLS]最後是該講一講這本書詭異到極點的等級設定，想我當初還認真研究一下他的規則，老實說，在豆宗之前，豆破的等級還是很有規則的[SEP]
predict:驚悚樂園 true:鬥破蒼穹
[CLS]吳磊弟弟也將會在結束現在的拍攝工作之後，投入到新劇的拍攝之中[SEP]
predict:驚悚樂園 true:鬥破蒼穹
[CLS]就在他即將絕望的時候，一縷靈魂從他手上的戒指裡浮現，一扇全新的大門在面前開啟，經過艱苦修煉最終成就輝煌的故事，是一部玄幻愛情小說，作品推出之後大獲歡迎[SEP]
predict:琥珀之劍 true:鬥破蒼穹
[CLS]我覺得只有傻子皇帝才可能投降永興這一脈本來就是造反得來皇位若真的又被拿回去估計全部皇室都會被殺光想想如果明朝建文帝後代拿回皇位有可能不清洗朱棣的後代嗎？[SEP]
predict:琥珀之劍 true:大奉打更人
[CLS]虽然让他跑了34大家好我是萧战[SEP]
predict:全職高手 true:鬥破蒼穹
[CLS]並且吳磊弟弟是童星出身，有著很多演戲經驗，雖然只有17歲，不過吳磊弟弟可是演藝圈裡的前輩級的人物了，出演了很多作品[SEP]
predict:全職高手 true:鬥破蒼穹
[CLS]照道理就該對帝玉加強戒備了呀[SEP]

[CLS]打臉高手兼軟飯一哥[SEP]
predict:修真聊天群 true:鬥破蒼穹
[CLS][UNK]我已經來了[SEP]
predict:修真聊天群 true:詭秘之主
[CLS]西園寺世界都還知道要補一刀，可憐一代毒仙就這樣傻呼呼地被拐走了，悲乎哀乎痛乎[SEP]
predict:驚悚樂園 true:鬥破蒼穹
[CLS]這對大便來說，根本是最賺的無本生意啊[SEP]
predict:修真聊天群 true:仙逆
[CLS]讓你走了不少彎路還要報恩??土豆你[SEP]
predict:修真聊天群 true:鬥破蒼穹
[CLS]臉上露出掙扎之色[SEP]
predict:修真聊天群 true:仙逆
[CLS]土豆無奈道:穿越者，人恆穿之，打臉者，人必打臉之[SEP]
predict:修真聊天群 true:鬥破蒼穹
[CLS]我覺得只有傻子皇帝才可能投降永興這一脈本來就是造反得來皇位若真的又被拿回去估計全部皇室都會被殺光想想如果明朝建文帝後代拿回皇位有可能不清洗朱棣的後代嗎？[SEP]
predict:琥珀之劍 true:大奉打更人
[CLS]他寫的《變色龍》、《一碗清湯蕎麥面》、《最后一課》等等多次被選作語文課本教材[SEP]
predict:驚悚樂園 true:鬥破蒼穹
[CLS][UNK][UNK]好的，老師[SEP]
predict:詭秘之主 true:驚悚樂園
[CLS]第一關失敗後連過四關[SEP]
predict:修真聊天群 true:仙逆
[CLS][UNK]我就知道[SEP]
predict:琥珀之劍 true:全職高手
[CLS]會不會是燭離阿?露臉之後[SEP]
predict:修真聊天群 true:鬥破蒼穹
[CLS]"鳳清兒的眼淚"這一章是本人寫的嗎[SEP]
predict:修真聊天群 true:鬥破蒼穹
[CLS][UNK]沒意思[SEP]
predict:琥珀之劍 true:詭秘之主
[CLS]沒忘記的舉手![SEP]
predict:修真聊天群 true:琥珀之劍
[CLS]大略的整理下可能相關的線索，有些只是憑感覺列出的，可能有關，也可能無關：1[SEP]
predict:大奉打更人 true:琥珀之劍
[CLS]這種感覺，唯有長久地身居高位[SEP]
predict:全職高手 true:仙逆
[CLS]最近說實在比較沒爆點[S

[CLS]我只希望土豆能好好寫新書，別再折騰炎哥了，看一個兩年前充滿志向的大好青年變成現在這個動不動就滅人全家的劊子手，心理挺難過的[SEP]
predict:驚悚樂園 true:鬥破蒼穹
[CLS][UNK]哎哎，你别哭了，是你自己說可以的[SEP]
predict:全職高手 true:大奉打更人
[CLS]除了把九哥可能會掛掉，然後能名正言順吃掉九妹的梗埋下[SEP]
predict:驚悚樂園 true:鬥破蒼穹
[CLS]奢侈就奢侈吧，總比亂了套要好[SEP]
predict:修真聊天群 true:全職高手
[CLS]肺炎的老習慣又來了視角突然跳到帕西米拉那邊看來要跟著一群平民大冒險是知道這傢伙很愛用這手法層層鋪陳可是用多了就有點讓人煩躁阿[SEP]
predict:全職高手 true:琥珀之劍
[CLS]另外，謝謝大家的月票支持，打賞感謝章留到完本的時候吧，沒幾天了[SEP]
predict:修真聊天群 true:大奉打更人
[CLS]寶物我的就是我的，你的也是我的，全部都是我的6[SEP]
predict:修真聊天群 true:鬥破蒼穹
[CLS]而且安排飯聖打雷族小輩的臉，果然完全是多餘的[SEP]
predict:修真聊天群 true:鬥破蒼穹
[CLS]了吧還摸一摸心臟咧個人預測敵方半聖或豆聖只是為了把消炎打死然後便可以讓他在昏迷間加速煉化之類的[SEP]
predict:修真聊天群 true:鬥破蒼穹
[CLS]燃燒的學園-殞落心炎2[SEP]
predict:琥珀之劍 true:鬥破蒼穹
[CLS]最近看到起點一本都市小說簡介是重生與地球相似的異界，腦中平白無故多了一個搜索引擎[SEP]
predict:琥珀之劍 true:鬥破蒼穹
[CLS]無奈但也接受[SEP]
predict:琥珀之劍 true:全職高手
[CLS]不能說這方法有錯，但讀起來跟姐姐是大明星跟妖二代相比，好像缺少鋪陳跟一些時間讓情緒起伏，就一直甩出下一段強行讓讀者跟上[SEP]
predict:驚悚樂園 true:大奉打更人
[CLS]4兩名高級被幹掉接著清場[SEP]
predict:全職高手 true:鬥破蒼穹
[CLS]～～～##～##～##～##～##～##～##～##～##～##～##～##～##～##～##～##～##～##～##～##～##～##～##～##～#

[CLS]眾人齊奸笑:消炎，謝禮這就來了~~~~消炎臉無人色，轉頭向土豆說:這到底怎麼一回事[SEP]
predict:修真聊天群 true:鬥破蒼穹
[CLS]實在是不希望這部作品就此完結啊！[SEP]
predict:修真聊天群 true:全職高手
[CLS]都快結束了還可以跑出一堆新角色不過說歸說還是會看下去[SEP]
predict:全職高手 true:鬥破蒼穹
[CLS]三兩下就被套完話了[SEP]
predict:修真聊天群 true:琥珀之劍
[CLS]剛才是一場夢？可我感覺，感覺是那樣的真實[SEP]
predict:修真聊天群 true:詭秘之主
[CLS]，她的兒女都是人造人除了皇長子是她哥哥生的又是合法蘿莉，不舉多不收進後宮嗎?[SEP]
predict:大奉打更人 true:琥珀之劍
[CLS]此時的他，早就忘記了自已曾經是一個騙子[SEP]
predict:修真聊天群 true:仙逆
[CLS]斗篷之下，沒有身體存在，一片幽暗[SEP]
predict:仙逆 true:詭秘之主
[CLS]為什麼我看到這幾句話[SEP]
predict:大奉打更人 true:鬥破蒼穹
[CLS]』看鬥破根本就像在看jo##ke板嘛xd毒到這麼好笑也算土豆功力深厚阿!![SEP]
predict:全職高手 true:鬥破蒼穹
[CLS]請問我對『鬼喊抓鬼』有這感覺是正常的嗎？還是只是因為電波不合[SEP]
predict:修真聊天群 true:驚悚樂園
[CLS]而且很顯然炎帝覺得自己是天才研發新招，算自己的努力[SEP]
predict:驚悚樂園 true:鬥破蒼穹
[CLS]身為多年前入坑玄幻小說，專看爽書的毒抗道友也是深有同感，這些年應該看了有3[SEP]
predict:修真聊天群 true:鬥破蒼穹
[CLS]防雷頁[UNK]的熊[SEP]
predict:琥珀之劍 true:仙逆
[CLS]是他死在監獄裡，無法閉上的眼睛[SEP]
predict:修真聊天群 true:大奉打更人
[CLS]但我自己個人而言比較喜歡他前兩本書的節奏，但作者為了$$妥協也沒錯[SEP]
predict:驚悚樂園 true:大奉打更人
[CLS]有沒有人推薦主角是從低端練到高端，然後會和女主有比較多的互動的書啊？有點類似噩盡島i，洛年和懷真那樣的感覺，或是無職轉生中的盧

[CLS][SEP]
predict:修真聊天群 true:大奉打更人
[CLS]猜拳猜輸的阿蒙會變祭品不過有一點疑問只有我覺得以一個能困住學徒天使之王的封印來說解封代價只要三個半神好像低的有點太不合比例了吧感覺裡面有鬼[SEP]
predict:琥珀之劍 true:詭秘之主
[CLS]要開新副本了～瘋子是掌尊之主戰老鬼想撿骨化凡again開始？[SEP]
predict:修真聊天群 true:仙逆
[CLS]他寫的《變色龍》、《一碗清湯蕎麥面》、《最后一課》等等多次被選作語文課本教材[SEP]
predict:驚悚樂園 true:鬥破蒼穹
[CLS][UNK]哈哈[SEP]
predict:修真聊天群 true:全職高手
[CLS]這邊大戰完之後[SEP]
predict:琥珀之劍 true:鬥破蒼穹
[CLS]今天還有兩張仙逆本月票數：626##9票今天：230票斗破本月票数：62##33票今天：70##3票一天被追上呀[SEP]
predict:修真聊天群 true:仙逆
[CLS]請依照劇情進度自行帶入○○人名、勢力名、ｘ##ｘ豆豆等級這毒豆什麼時候要完結？[SEP]
predict:修真聊天群 true:鬥破蒼穹
[CLS]所以肯定是给你用的[SEP]
predict:修真聊天群 true:大奉打更人
[CLS]猜拳猜輸的阿蒙會變祭品不過有一點疑問只有我覺得以一個能困住學徒天使之王的封印來說解封代價只要三個半神好像低的有點太不合比例了吧感覺裡面有鬼[SEP]
predict:琥珀之劍 true:詭秘之主
[CLS]算是經典作品吧？苦撐一段時間後棄坑了[SEP]
predict:修真聊天群 true:鬥破蒼穹
[CLS]數據的抗爭就滿好看的，故事也完整多了[SEP]
predict:修真聊天群 true:驚悚樂園
[CLS]同時網友們對於電視劇的女主角人選也很關注，紛紛推薦了自己心目中覺得適合的演員[SEP]
predict:全職高手 true:鬥破蒼穹
[CLS]應該是在做夢~感覺快回去了這幾章應該比當初化凡更好看一點內容大家自己看感覺會比較好~~不過還是會覺得當初李那裏寫的實在是有點少xd[SEP]
predict:驚悚樂園 true:仙逆
[CLS][UNK]當心，那傢伙心黑的很[SEP]
predict:修真聊天群 true:全職高手
[CLS]他寫

[CLS]不知道有沒有推薦查案型古代歷史小說穿越開掛也ok[SEP]
predict:詭秘之主 true:大奉打更人
[CLS]半空中突然出現數百道裂縫，每道裂縫中都有一人緩緩走出[SEP]
predict:仙逆 true:鬥破蒼穹
[CLS][UNK][UNK]哼[SEP]
predict:驚悚樂園 true:全職高手
[CLS]盯了屍體10天ㄟ[SEP]
predict:修真聊天群 true:鬥破蒼穹
[CLS]林動楊洋吳尊小炎林動的寵物化形怎麼演林瑯天阿記得是一個很悲慘的角色被主角從頭虐到尾[SEP]
predict:修真聊天群 true:鬥破蒼穹
[CLS]小克你前世一定是客家人[SEP]
predict:修真聊天群 true:詭秘之主
[CLS]之前看到奪基附近就啃不下去了，看到這篇文這幾天啃完它了感覺接近200章附近毒素有減少一點，化凡階段開始好看[SEP]
predict:琥珀之劍 true:仙逆
[CLS]但是回頭看看都還覺得寫得有些味道，好不容易到的豆聖豆帝了，千招百招就剩兩招，c4和自爆[SEP]
predict:全職高手 true:鬥破蒼穹
[CLS]一瞬間像是蒼老了數十歲[SEP]
predict:琥珀之劍 true:大奉打更人
[CLS]現在是誰的本源多，誰說話就比較大聲的時代[SEP]
predict:修真聊天群 true:仙逆
[CLS]我一氣呵成幾天時間看完，感覺場面宏大，情節編排合理，環環緊扣[SEP]
predict:修真聊天群 true:大奉打更人
[CLS]ps：這本書還有三四天完本，大家這個月就不用給我投月票了[SEP]
predict:修真聊天群 true:大奉打更人
[CLS]ex:遠古八族的石族還有靈族[SEP]
predict:仙逆 true:鬥破蒼穹
[CLS]並不覺得這個完結太過倉促，也不覺得這之間的過程太過沉長，或許可以做出各細緻或更簡潔的戰鬥，但那些都不會是今天的全職高手[SEP]
predict:驚悚樂園 true:全職高手
[CLS][SEP]
predict:修真聊天群 true:大奉打更人
[CLS]斗篷之下，沒有身體存在，一片幽暗[SEP]
predict:仙逆 true:詭秘之主
[CLS]謎一樣的蘑菇之王[SEP]
predict:琥珀之劍 true:詭秘之主
[CLS]然而到中期開始就漸漸轉型

[CLS]但是，快樂的時光總是短暫的，正所謂春宵一刻值千金，土豆的寫作水準就像那春宵一樣一去不復返[SEP]
predict:驚悚樂園 true:鬥破蒼穹
[CLS]下一次爆更要等生小孩了嗎？[SEP]
predict:修真聊天群 true:驚悚樂園
[CLS]npc大軍登場！"目前為止"的攻擊方式都沒有針對玩家的跡象，玩家只有受波及才會受到傷害，但還沒講傷害多少不知道如果玩家改變npc陣型會不會使ai也跟著改變[SEP]
predict:驚悚樂園 true:全職高手
[CLS]這章有異世邪君的feel[SEP]
predict:修真聊天群 true:仙逆
[CLS]說真的我受夠這個食屍鬼傳奇了越看越噁心動不動就那邊xx遺骨xx精血[SEP]
predict:大奉打更人 true:鬥破蒼穹
[CLS]多少生靈繁衍[SEP]
predict:大奉打更人 true:仙逆
[CLS]詭祕的完結成為一個閱文事件的代名詞[SEP]
predict:修真聊天群 true:詭秘之主
[CLS]自己回自己文[SEP]
predict:修真聊天群 true:鬥破蒼穹
[CLS]對不起風尊者，我不是說你是廢物，我是指所有的尊者都是廢物!想當初我看到炎哥找到風尊者時多感動，想說終於雨過天晴有個強大的靠山了[SEP]
predict:修真聊天群 true:鬥破蒼穹
[CLS]他是在用自己的安危，減輕你們的傷亡[SEP]
predict:驚悚樂園 true:大奉打更人
[CLS]更了!!月更一回!!![SEP]
predict:全職高手 true:琥珀之劍
[CLS][UNK]又給你上了一課[SEP]
predict:修真聊天群 true:全職高手
[CLS]土豆無奈道:穿越者，人恆穿之，打臉者，人必打臉之[SEP]
predict:修真聊天群 true:鬥破蒼穹
[CLS]這次事件之後會不會想到詭祕[SEP]
predict:琥珀之劍 true:詭秘之主
[CLS]前情提要：不舉多找到小公主，接下就當跑腿小弟了～[SEP]
predict:大奉打更人 true:琥珀之劍
[CLS]順便問一下這個遊戲的陷阱是看得見還看不見?[SEP]
predict:驚悚樂園 true:全職高手
[CLS]在最新一章中提到[SEP]
predict:琥珀之劍 true:仙逆
[CLS]從那裡離開後，我再也沒

[CLS]／[UNK]д′／~[UNK]誰來告訴我現在演到哪裡啊，都忘光了xd好像是去救下屬是吧?這張算是過渡張，總之不懶惰發現歷史可能跟遊戲中有很大的差異，然後新紀元的到來提前了[SEP]
predict:驚悚樂園 true:琥珀之劍
[CLS]他寫的《凡人修仙傳》、《斗破蒼穹》連續蟬聯月票榜[SEP]
predict:修真聊天群 true:鬥破蒼穹
[CLS]烏賊完本感言上第八部都是已經考慮好的[SEP]
predict:修真聊天群 true:詭秘之主
[CLS]老夫甘願創出那三千之外地．．．．．道！"========================================="老夫[SEP]
predict:修真聊天群 true:仙逆
[CLS]不然推敲就有很多不合理[SEP]
predict:大奉打更人 true:鬥破蒼穹
[CLS]如果寫完前做不到，那書就會重置除了基本規則外的所有新規則[SEP]
predict:驚悚樂園 true:詭秘之主
[CLS]他寫的《達芬奇密碼》、《傲慢與偏見》等名著，被多次改變為電影[SEP]
predict:詭秘之主 true:鬥破蒼穹
[CLS]狹小的夾縫中念氣也張揚不開，只能朝著有限的空間迸散[SEP]
predict:驚悚樂園 true:全職高手
[CLS]連忙低聲開口:『[SEP]
predict:大奉打更人 true:仙逆
[CLS][UNK]=======[UNK]qq##qq##qq##qq大帝托孤了不知道該說什麼好一代帝王這樣真的慘可以說小克路好走都是他先舖的[SEP]
predict:全職高手 true:詭秘之主
[CLS]另外，我將一部分碎片封存在同一個地方，作為我遺產的一部分留給我的後人[SEP]
predict:修真聊天群 true:琥珀之劍
[CLS]說完題外話，回到書本身[SEP]
predict:大奉打更人 true:詭秘之主
[CLS]這份心意太重了[SEP]
predict:修真聊天群 true:大奉打更人
[CLS]太經典了這句[SEP]
predict:修真聊天群 true:仙逆
[CLS]一記響亮的法槌敲擊聲過后[SEP]
predict:修真聊天群 true:驚悚樂園
[CLS]人物總體等級又提高了上期出來的兩個魂族強到主角無法抗衡的鬥剩過一期就被捏爆了接下來呢？[SEP]
p

[CLS]死不知道多少人[SEP]
predict:大奉打更人 true:詭秘之主
[CLS][UNK]====三個重點[SEP]
predict:驚悚樂園 true:詭秘之主
[CLS]記得還有搶到一顆九品化形玄丹,也忘了[SEP]
predict:修真聊天群 true:鬥破蒼穹
[CLS]到了這個境界隨便必個關都是百八十年的現在不過兩年就在那邊擔心是在擔心個x[SEP]
predict:全職高手 true:鬥破蒼穹
[CLS][UNK]你還好吧[SEP]
predict:驚悚樂園 true:大奉打更人
[CLS]，她的兒女都是人造人除了皇長子是她哥哥生的又是合法蘿莉，不舉多不收進後宮嗎?[SEP]
predict:大奉打更人 true:琥珀之劍
[CLS]愛看逆天派的這本蠻不錯的，只是走到後期劇情也是一直重複如果不是主角光環開超大，一般人逆天到一半就被打死了吧[SEP]
predict:修真聊天群 true:仙逆
[CLS]好像是衝著e##04,看你什麼時候能毒死我[SEP]
predict:驚悚樂園 true:鬥破蒼穹
[CLS]你終究是要回來救援的[SEP]
predict:修真聊天群 true:大奉打更人
[CLS][UNK]嗯[SEP]
predict:修真聊天群 true:驚悚樂園
[CLS]就是要問天一聲[SEP]
predict:大奉打更人 true:仙逆
[CLS]沒有女性投懷送抱的滅掉也沒差[SEP]
predict:修真聊天群 true:鬥破蒼穹
[CLS]肺炎這一章又出包bug了而且要修這個bug需要大幅修改這一章的劇情聖堂騎士的荊棘冠冕這個技能，原本主動開啟的效用是攻擊力大幅提升30分鐘，效用結束後角色防禦[SEP]
predict:全職高手 true:琥珀之劍
[CLS]一瞬間像是蒼老了數十歲[SEP]
predict:琥珀之劍 true:大奉打更人
[CLS]最後我還是要用我血淚教訓奉勸其他人[SEP]
predict:琥珀之劍 true:鬥破蒼穹
[CLS]到現在剛好一個月適逢過年[SEP]
predict:修真聊天群 true:全職高手
[CLS]是之前某仙界的地圖導航員嗎？[SEP]
predict:修真聊天群 true:仙逆
[CLS]我做了一個致命的陷阱在等你[SEP]
predict:修真聊天群 true:全職高手
[CL

[CLS]完全把消炎藥寫成一個耗呆1[SEP]
predict:驚悚樂園 true:鬥破蒼穹
[CLS]如果只有幾章還好，可是一連五章[SEP]
predict:修真聊天群 true:驚悚樂園
[CLS]剛玩完ea的俠隱閣劇情不錯啊，河洛你也快更新-敲碗中題外話，我自己是比較喜歡廢土拉，廢土寫得好的少阿[SEP]
predict:驚悚樂園 true:詭秘之主
[CLS]那就只有一種可能了[SEP]
predict:琥珀之劍 true:驚悚樂園
[CLS]煩請各位推薦啦~[SEP]
predict:修真聊天群 true:鬥破蒼穹
[CLS]因為在一個網站看到十大好看修真小說，選了半天就點了仙逆開始看[SEP]
predict:修真聊天群 true:仙逆
[CLS][UNK]哈哈[SEP]
predict:修真聊天群 true:全職高手
[CLS]身為多年前入坑玄幻小說，專看爽書的毒抗道友也是深有同感，這些年應該看了有3[SEP]
predict:修真聊天群 true:鬥破蒼穹
[CLS]而且這部只有主角能越階不只越級還越階打人，其他人每越一級就差的天差地遠[SEP]
predict:修真聊天群 true:鬥破蒼穹
[CLS]反正都已經多收了[SEP]
predict:修真聊天群 true:仙逆
[CLS]至少，在三年之約之前，這本書還是有值得一看的地方[SEP]
predict:琥珀之劍 true:鬥破蒼穹
[CLS]最近看到起點一本都市小說簡介是重生與地球相似的異界，腦中平白無故多了一個搜索引擎[SEP]
predict:琥珀之劍 true:鬥破蒼穹
[CLS]奪妻魂雙旗於一身簡直就準備死得不能再死有夠廉價的boss[SEP]
predict:全職高手 true:仙逆
[CLS]看到這裡我就差不多不行了，很少看書看到那麼氣的，後面留給各板友補充[SEP]
predict:修真聊天群 true:鬥破蒼穹
[CLS]話說回來，四叔呢？我是四叔黨的xddd##d[SEP]
predict:大奉打更人 true:仙逆
[CLS]拋沙？聖光熄滅了[SEP]
predict:修真聊天群 true:全職高手
[CLS]我便去她房間幫忙尋找這一找就找到了黃昏[SEP]
predict:琥珀之劍 true:大奉打更人
[CLS]神魔賭局的部分有兩個可惜的地方，第一個參與的神魔形象很模

In [32]:
"""
#plot
#plt.xlim(0,2200 ) # 設定 x 軸座標範圍
import matplotlib.pyplot as plt
plt.ylim(0, 1) # 設定 y 軸座標範圍

plt.xlabel('epoch', fontsize="10") # 設定 x 軸標題內容及大小
plt.ylabel('Accuracy', fontsize="10") # 設定 y 軸標題內容及大小
plt.title('Accuracy in use replace data (dropout=0.1)(8 movie prediction)', fontsize="18") # 設定圖表標題內容及大小

plt.plot(range(len(train_acc_list)),train_acc_list,marker='o', color='blue',label="Training Accuracy")
plt.plot(range(len(val_acc_list)),val_acc_list,marker='o', color='red',label="Validation Accuracy")
#for x,y in zip(range(len(val_acc_list)),val_acc_list): 
#    plt.text(x, y+0.01, "({}, {:.3f})".format(x,y))

plt.legend()
fig = plt.gcf()
fig.set_size_inches(15, 10)
plt.savefig("Movie Accuracy use replace data (dropout=0.1).png",dpi=100)

plt.clf()
"""

'\n#plot\n#plt.xlim(0,2200 ) # 設定 x 軸座標範圍\nimport matplotlib.pyplot as plt\nplt.ylim(0, 1) # 設定 y 軸座標範圍\n\nplt.xlabel(\'epoch\', fontsize="10") # 設定 x 軸標題內容及大小\nplt.ylabel(\'Accuracy\', fontsize="10") # 設定 y 軸標題內容及大小\nplt.title(\'Accuracy in use replace data (dropout=0.1)(8 movie prediction)\', fontsize="18") # 設定圖表標題內容及大小\n\nplt.plot(range(len(train_acc_list)),train_acc_list,marker=\'o\', color=\'blue\',label="Training Accuracy")\nplt.plot(range(len(val_acc_list)),val_acc_list,marker=\'o\', color=\'red\',label="Validation Accuracy")\n#for x,y in zip(range(len(val_acc_list)),val_acc_list): \n#    plt.text(x, y+0.01, "({}, {:.3f})".format(x,y))\n\nplt.legend()\nfig = plt.gcf()\nfig.set_size_inches(15, 10)\nplt.savefig("Movie Accuracy use replace data (dropout=0.1).png",dpi=100)\n\nplt.clf()\n'

In [44]:
model.eval()
test_text = "[CLS]裡面有旅法師相關[SEP]"  # 盡量湊到6個字以上 ex:xx到底是甚麼
tokens = tokenizer.tokenize(test_text)
ids = tokenizer.convert_tokens_to_ids(tokens)

id_tensor = torch.tensor([ids]).to(device)
print(test_text)
print(tokens[:])
print(ids[:])
k = 8
with torch.no_grad():
    outputs = model(input_ids=id_tensor)
    prediction = outputs[0]
    probs, indices = torch.topk(torch.softmax(prediction, -1), k)
    probs = probs.tolist()
    indices = indices.tolist()
    probs = probs[0]
    indices = indices[0]
for i, (indice, p) in enumerate(zip(indices, probs), 1):
    label = inv_label_map[indice]
    print("Top {} ({:2}%)：{}".format(i, int(p * 100), label[:10]))

[CLS]裡面有旅法師相關[SEP]
['[CLS]', '裡', '面', '有', '旅', '法', '師', '相', '關', '[SEP]']
[101, 6174, 7481, 3300, 3180, 3791, 2374, 4685, 7302, 102]
Top 1 (99%)：琥珀之劍
Top 2 ( 0%)：詭秘之主
Top 3 ( 0%)：修真聊天群
Top 4 ( 0%)：全職高手
Top 5 ( 0%)：大奉打更人
Top 6 ( 0%)：仙逆
Top 7 ( 0%)：驚悚樂園
Top 8 ( 0%)：鬥破蒼穹


In [34]:
def SeriesPrediction(text,model,tokenizer):
    test_text="[CLS]"+text+"[SEP]"   #盡量湊到6個字以上 ex: xx到底是甚麼
    tokens = tokenizer.tokenize(test_text)
    ids = tokenizer.convert_tokens_to_ids(tokens)

    id_tensor = torch.tensor([ids]).to(device)
    print(text)
    print(ids[:])
    k=8
    with torch.no_grad():
        outputs = model(input_ids=id_tensor)     
        prediction = outputs[0]
        probs, indices = torch.topk(torch.softmax(prediction, -1), k)
        probs=probs.tolist()
        indices=indices.tolist()
        probs=probs[0]
        indices=indices[0]
    for i, (indice, p) in enumerate(zip(indices, probs), 1):
        label=inv_label_map[indice]
        print("Top {} ({:2}%)：{}".format(i, int(p * 100), label[:10]), '...')

In [35]:
#text="姐姐其實是自己的媽媽"
text="託妹妹偽造名校的學校"

MoviePrediction(text,model,tokenizer)

NameError: name 'MoviePrediction' is not defined

In [ ]:
tokenizer.convert_tokens_to_ids("_")